In [1]:
import numpy as np

# ==== Dữ liệu ====
X = np.array([
    [1, 9],
    [2, 6],
    [2, 8],
    [3, 9],
    [2, 2],
    [3, 3],
    [4, 1],
    [5, 2],
    [5, 4],
    [8, 1]
], dtype=float)

y = np.array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0], dtype=float)

N, d = X.shape

# ==== Tham số ====
w = np.zeros(d)   # w(0) = [0,0]
b = 0.0           # b(0) = 0
eta = 1e-3        # tốc độ học
epochs = 100000   # số vòng lặp

# ==== Hàm sigmoid ====
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# ==== Thuật toán Gradient Descent ====
for t in range(epochs):
    z = X.dot(w) + b
    p = sigmoid(z)             # xác suất dự báo
    error = p - y              # sai số
    grad_w = (X.T.dot(error)) / N
    grad_b = np.mean(error)
    w -= eta * grad_w
    b -= eta * grad_b

# ==== Kết quả ====
print("Tham số w =", w)
print("Tham số b =", b)

# 1) Xác suất điểm (6,2) thuộc lớp 1
pt = np.array([6.0, 2.0])
prob_pt = sigmoid(pt.dot(w) + b)
print("Xác suất P(y=1 | (6,2)) =", prob_pt)

# 2) Đường phân tách: w1*x1 + w2*x2 + b = 0
# => x2 = (-w1/w2)*x1 - b/w2
slope = -w[0] / w[1]
intercept = -b / w[1]
print("Phương trình đường phân tách: x2 = %.3f * x1 + %.3f" % (slope, intercept))


Tham số w = [-2.85614925  1.72440695]
Tham số b = -0.8893545389345123
Xác suất P(y=1 | (6,2)) = 4.6676655260517145e-07
Phương trình đường phân tách: x2 = 1.656 * x1 + 0.516


In [6]:
import numpy as np

# ======================
# 1. DỮ LIỆU
# ======================
X = np.array([[1,9],[2,6],[2,8],[3,9],[2,2],
              [3,3],[4,1],[5,2],[5,4],[8,1]], dtype=float)
y = np.array([1,1,1,1,0,0,0,0,0,0], dtype=float)
N, d = X.shape   # N=10, d=2 (số mẫu, số đặc trưng)

# Thêm cột 1 để gom bias vào vector tham số (theta = [w1, w2, b])
X_tilde = np.hstack([X, np.ones((N,1))])

# ======================
# 2. CÁC HÀM CƠ BẢN
# ======================
def sigmoid(z):
    """Hàm sigmoid: σ(z) = 1/(1+exp(-z))"""
    return 1.0 / (1.0 + np.exp(-z))

def loss(theta, lambda_reg=1e-2):
    """
    Hàm mất mát (Negative log-likelihood) có thêm L2 regularization.
    Chỉ regularize w, không regularize bias.
    """
    z = X_tilde.dot(theta)
    p = sigmoid(z)
    w = theta[:-1]
    nll = -np.mean(y*np.log(p+1e-12) + (1-y)*np.log(1-p+1e-12))
    reg = 0.5 * lambda_reg * np.dot(w, w)
    return nll + reg

def grad_and_hess(theta, lambda_reg=1e-2):
    """
    Tính gradient và Hessian tại theta.
    Gradient: g = 1/N * X^T (p - y) + lambda * w
    Hessian: H = 1/N * X^T S X + lambda * I (trên phần w)
    """
    z = X_tilde.dot(theta)
    p = sigmoid(z)
    grad = (X_tilde.T.dot(p - y)) / N
    # chỉ regularize w (không regularize bias)
    reg_vec = np.concatenate([lambda_reg * theta[:-1], [0.0]])
    grad += reg_vec
    # ma trận S = diag(p*(1-p))
    S = p * (1 - p)
    H = (X_tilde.T.dot(np.diag(S)).dot(X_tilde)) / N
    # thêm lambda lên đường chéo phần w
    H[:d, :d] += lambda_reg * np.eye(d)
    return grad, H

# ======================
# 3. THUẬT TOÁN NEWTON-RAPHSON
# ======================
max_iter = 1000
tol = 1e-6
theta = np.zeros(d+1)  # khởi tạo w=(0,0), b=0
converged = False

for k in range(max_iter):
    grad, H = grad_and_hess(theta)
    # Giải hệ tuyến tính H * delta = grad
    try:
        delta = np.linalg.solve(H, grad)
    except np.linalg.LinAlgError:
        delta = np.linalg.pinv(H).dot(grad)
    # Cập nhật: theta_new = theta - delta
    theta_new = theta - delta
    # Điều kiện dừng
    if np.linalg.norm(theta_new - theta) < tol:
        theta = theta_new
        converged = True
        break
    theta = theta_new

# ======================
# 4. KẾT QUẢ CUỐI CÙNG
# ======================
w = theta[:-1]
b = theta[-1]

print("Thuật toán hội tụ:", converged, "sau", k+1, "vòng lặp")
print("Tham số cuối cùng:")
print("   w =", w)
print("   b =", b)
print("Giá trị hàm mất mát =", loss(theta))

# ======================
# 5. DỰ BÁO ĐIỂM (3,7)
# ======================
pt = np.array([3.0, 7.0])
prob_pt = sigmoid(pt.dot(w) + b)
print("\nXác suất P(y=1 | (3,7)) =", prob_pt)
print("Dự đoán lớp =", int(prob_pt >= 0.5))

# ======================
# 6. ĐƯỜNG PHÂN TÁCH
# ======================
# Công thức: b + w1*x1 + w2*x2 = 0  =>  x2 = -(w1/w2)x1 - b/w2
if abs(w[1]) > 1e-12:
    slope = -w[0]/w[1]
    intercept = -b/w[1]
    print("\nPhương trình đường phân tách:")
    print("   x2 = {:.6f} * x1 + {:.6f}".format(slope, intercept))
else:
    print("\nĐường phân tách gần như thẳng đứng tại x1 = {:.6f}".format(-b/w[0]))


Thuật toán hội tụ: True sau 9 vòng lặp
Tham số cuối cùng:
   w = [-0.96960175  1.80486752]
   b = -6.12177762654924
Giá trị hàm mất mát = 0.03355928728594654

Xác suất P(y=1 | (3,7)) = 0.9734932056414295
Dự đoán lớp = 1

Phương trình đường phân tách:
   x2 = 0.537215 * x1 + 3.391816
